In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier

In [ ]:
import lightgbm as lg

In [ ]:
path = "../input/stanford-covid-vaccine/bpps/id_00073f8be.npy"
data_np = np.load(path)
plt.plot(data_np)
plt.show()

In [ ]:
path_np = "../input/stanford-covid-vaccine/bpps"
arr = os.listdir(path_np)
array = np.load(f'../input/stanford-covid-vaccine/bpps/{arr[30]}')
print(len(arr))
print(array.shape)

In [ ]:
test  = pd.read_json("../input/stanford-covid-vaccine/test.json", lines = True)
train = pd.read_json("../input/stanford-covid-vaccine/train.json", lines = True)

In [ ]:
test    = test.set_index('index')
train   = train.set_index('index')

In [ ]:
train.head(3)

In [ ]:
csv = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')

In [ ]:
print('========= train columns ==========')
print([c for c in train.columns])

print('========= test columns ==========')
print([c for c in test.columns])

In [ ]:
x = 25
diagram = plt.figure(figsize = (20,20))
for i in range(x):
    np_ex = np.load(f'../input/stanford-covid-vaccine/bpps/{arr[i]}')
    sub_plt = diagram.add_subplot(5,5,i+1)
    sub_plt.imshow(np_ex)

In [ ]:
Counter(train['sequence'].values[1])

In [ ]:
Counter(train['predicted_loop_type'].values[1])

In [ ]:
def count(c):
    c['A'] = c['sequence'].apply(lambda s: s.count('A'))
    c['G'] = c['sequence'].apply(lambda s: s.count('G'))
    c['U'] = c['sequence'].apply(lambda s: s.count('U'))
    c['C'] = c['sequence'].apply(lambda s: s.count('C'))
    return c

In [ ]:
def loop_count(p):
    p['E'] = p['predicted_loop_type'].apply(lambda s: s.count('E'))
    p['S'] = p['predicted_loop_type'].apply(lambda s: s.count('S'))
    p['H'] = p['predicted_loop_type'].apply(lambda s: s.count('H'))
    p['B'] = p['predicted_loop_type'].apply(lambda s: s.count('B'))
    p['M'] = p['predicted_loop_type'].apply(lambda s: s.count('M'))
    return p

In [ ]:
count_train = count(train)
count_test  = count(test)

In [ ]:
loop_count_train = loop_count(train)
loop_count_test  = loop_count(test)

In [ ]:
train['mean_reactivity'] = train['reactivity'].apply(lambda x: np.mean(x))
train['mean_deg_Mg_pH10'] = train['deg_Mg_pH10'].apply(lambda x: np.mean(x))
train['mean_deg_Mg_50C'] = train['deg_Mg_50C'].apply(lambda x: np.mean(x))

In [ ]:
train['median_reactivity'] = train['reactivity'].apply(lambda x: np.median(x))
train['median_deg_Mg_pH10'] = train['deg_Mg_pH10'].apply(lambda x: np.median(x))
train['median_deg_Mg_50C'] = train['deg_Mg_50C'].apply(lambda x: np.median(x))

In [ ]:
train

In [ ]:
train.columns

In [ ]:
train

In [ ]:
test

In [ ]:
seq_col  = [c for c in train.columns if 'sequence_' in c]
loop_col = [c for c in train.columns if 'predicted_loop_type_' in c]
remaining = ['A','G','U','C',
          'E','S','H','B','M']
tot = seq_col + loop_col + remaining

In [ ]:
#with median

In [ ]:
for t in ['reactivity','deg_Mg_pH10','deg_Mg_50C']:

    X = train[tot]
    y = train[f'median_{t}']
    X_test = test[tot]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

    reg = lg.LGBMRegressor()
    reg.fit(X_train, y_train,
            eval_set=(X_val, y_val),
           early_stopping_rounds=100,
           verbose=100)

    test[f'median_{t}_pred'] = reg.predict(X_test)

In [ ]:
#with mean

In [ ]:
for t in ['reactivity','deg_Mg_pH10','deg_Mg_50C']:

    X = train[tot]
    y = train[f'mean_{t}']
    X_test = test[tot]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

    reg = lg.LGBMRegressor()
    reg.fit(X_train, y_train,
            eval_set=(X_val, y_val),
           early_stopping_rounds=100,
           verbose=100)

    test[f'mean_{t}_pred'] = reg.predict(X_test)

In [ ]:
test

In [ ]:
new_train = train

In [ ]:
new_train

In [ ]:
new_data = new_train.drop(columns='reactivity_error')
new_data = new_train.drop(columns='deg_error_Mg_pH10')
new_data = new_train.drop(columns='deg_Mg_50C')

In [ ]:
new_data

In [ ]:
pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')

In [ ]:
csv['id'] = 'id_' + csv['id_seqpos'].str.split('_', expand=True)[1]

# Merge my predicted average values
csv_new = csv. \
    drop(['reactivity','deg_Mg_pH10','deg_Mg_50C'], axis=1) \
    .merge(test[['id',
               'median_reactivity_pred',
               'median_deg_Mg_pH10_pred',
               'median_deg_Mg_50C_pred']] \
               .rename(columns={'median_reactivity_pred' : 'reactivity',
                                'median_deg_Mg_pH10_pred': 'deg_Mg_pH10',
                                'median_deg_Mg_50C_pred' : 'deg_Mg_50C'}
                      ),
         on='id',
        validate='m:1')

In [ ]:
one = csv_new[csv.columns]
one.drop(columns='id')



In [ ]:
one = one.drop(['id'],axis=1)

In [ ]:
one

In [ ]:
one[one.columns].to_csv('submission.csv')

In [ ]:
new_csv = csv

In [ ]:
csv['id'] = 'id_' + csv['id_seqpos'].str.split('_', expand=True)[1]

# Merge my predicted average values
new_csv = csv. \
    drop(['reactivity','deg_Mg_pH10','deg_Mg_50C'], axis=1) \
    .merge(test[['id',
               'mean_reactivity_pred',
               'mean_deg_Mg_pH10_pred',
               'mean_deg_Mg_50C_pred']] \
               .rename(columns={'mean_reactivity_pred' : 'reactivity',
                                'mean_deg_Mg_pH10_pred': 'deg_Mg_pH10',
                                'mean_deg_Mg_50C_pred' : 'deg_Mg_50C'}
                      ),
         on='id',
        validate='m:1')

In [ ]:
piece = new_csv[csv.columns]

In [ ]:
t